# Short example script for calculating the dimensionless linear power spectrum

$$\Delta(k_\rm{p},z=1.0) = k_\rm{p}^3 P_{\rm{lin}}(k_\rm{p},z=1.0)$$ at pivot scale $k_\rm{p} = 0.2\, h/\rm{Mpc}$

## Import modules, set up settings

In [5]:
import classy
from classy import Class
import numpy as np
import pandas as pd

#pivot scale + redshift
z_p = 1.0
k_p = 0.2 #our pivot scale, in h/Mpc units

#settings to configure CLASS computations
settings = {
    'output': 'mPk', #which quantities we want CLASS to compute
    'z_pk' : z_p,
    'non linear' : 'none',
    
    'P_k_max_1/Mpc':1.0 # CLASS uses 1/Mpc units
}

## Inspect CLASS version

NOTE: For Beyond-2pt submissions, should be ```v.2.9.4```

In [6]:
print(f"CLASS version: {classy.__version__}")

CLASS version: v2.9.4


## Compute ```norm``` == dimensionless linear matter power spectrum at $k_p$

### Define function(s)

In [7]:
def compute_norm(params,z_p=z_p):
    #initialize Class instance and set parameters
    cosmo = Class();
    cosmo.set(settings);
    cosmo.set(params)
    cosmo.compute()
    #print("Sigma8(z_pivot):")
    #print(cosmo.sigma(8./cosmo.h(),z_p)) # same as calling cosmo.sigma8
    #print("Corresponding A_s:")
    #print(cosmo.get_current_derived_parameters(['A_s']))

    #if CLASS used shooting method to determine A_s, check that it worked to better that 0.1% in sigma8
    #if 'sigma8' in params:
        #delta_sigma8 = np.abs((params['sigma8']-cosmo.sigma(8/cosmo.h(),0))/params['sigma8'])
        #assert delta_sigma8 < 1.e-3, f"A_s shooting method caused %.4f fractional bias in sigma8"%(delta_sigma8)

    #h = cosmo.h()
    #k_p_CLASS = k_p*h # convert to 1/Mpc units used by CLASS
    #norm = k_p**3*cosmo.pk(k_p_CLASS,z_p)*h**3 #factor h**3 to convert P(k) from 1/Mpc to h/Mpc units
    sigma8zp = cosmo.sigma(8./cosmo.h(),z_p)
    sigma8z0 = cosmo.sigma8()
    return sigma8zp, sigma8z0

### Compute ```norm``` and $A_s$, then add their columns to the new output files

This could be done much faster with bash scripts. Here I'm just trying to make it humanly readable using ```python```.

In [20]:
# Specify the input and output file paths

chain_param_path='/cobra/ptmp/minh/lefty_challenge/flatLCDM/box10/sampling_bO_shat/Eulerian_NUFFT/twolptsecond_nufft/'
chain_prefix='chain1a/sampling_bO_shat_2lptsecond_nufft_FOURIER_NGEul1024_L0.14_cube_logP_params_including_As_'

#chain_param_path='/cobra/ptmp/minh/lefty_challenge/flatLCDM/box1/sampling_bO/Eulerian_NUFFT/'
#chain_prefix='twolptsecond_nufft/kmax0125/chain1_fixed_highsigma_CLASS_largestep/sampling_bO_2lptsecond_nufft_FOURIER_NGEul1024_L0.175_cube_fixed_highsigma_CLASS_largestep_logP_params_including_As_'

input_files = []
output_files = []
nr_chains=2
for i in range(1,nr_chains+1):
    input_files.append(chain_param_path+chain_prefix + str(i) + '.txt')
    output_files.append(chain_param_path+chain_prefix+'sigma8z.' + str(i) + '.txt')

# Specify the *fixed* fiducial cosmological parameters as a dict

params = {
    #'h': 0.676225,
    'omega_b': 0.0222554,
    'omega_cdm': 0.118967,
    'n_s': 0.991533,
    'N_ur': 2.0328,
    'N_ncdm' : 1.0,
    'omega_ncdm': 0.00064420,
    '100*theta_s' : 1.041533,
    'recombination' : 'HyRec',
    'tau_reio' : 0.0544,
    'P_k_max_1/Mpc': 50.0,
    'k_pivot': 0.05,
}

for in_file, out_file in zip(input_files, output_files):
    data = np.genfromtxt(in_file, delimiter=None)
    params['A_s'] = data[0,-1]
    sigma8zp_0, sigma8z0_0 = compute_norm(params)
    sigma8zp = sigma8zp_0 * (data[:,2]/data[0,2])
    sigma8z0 = sigma8z0_0 * (data[:,2]/data[0,2])
    appended_data = np.column_stack((data, sigma8zp, sigma8z0))
    np.savetxt(out_file, appended_data, delimiter=' ')